# Austin Restaurant Data Analysis
An investigation into the relationship between visual content (restaurant photos) and customer ratings using Google Places and Cloud Vision APIs.

# Unstructured Data Analytics Final Project

### Gabriel Kinshuk, Carol Le, Vyshnavi Maringanti, Niladri Nath, Radha Pawar 

#### Using the Google Places API to get data about restaurants in Austin

Dependencies

In [13]:
# !pip install requests pandas

Imports

In [1]:
import requests
import json
import time
import pandas as pd
import math
from typing import List, Dict, Any, Tuple

Configuration

In [15]:
with open("google_places_api_key.txt", "r") as f:
    API_KEY = f.read().strip()

Scraper Class - We initially intended to scrape the data directly but found we could instead use the Google Places API, so this name is slightly misleading

In [40]:
class AustinRestaurantScraper:
    def __init__(self, api_key: str):
        self.api_key = api_key
        self.base_url = "https://maps.googleapis.com/maps/api/place"
        
    def generate_grid_points(self, bounds: Dict[str, float], radius_km: float = 1.5) -> List[Tuple[float, float]]:
        """
        Generate a grid of lat/lng points to cover Austin comprehensively.
        
        Args:
            bounds: Dict with 'north', 'south', 'east', 'west' coordinates
            radius_km: Search radius in kilometers (smaller = more coverage, more API calls)
        
        Returns:
            List of (lat, lng) tuples
        """
        # Convert radius to approximate degrees
        # 1 degree latitude ≈ 111 km
        # 1 degree longitude ≈ 111 km * cos(latitude) - at Austin's latitude ~96 km
        lat_step = (radius_km * 1.4) / 111  # 1.4x radius for overlap
        lng_step = (radius_km * 1.4) / 96
        
        points = []
        lat = bounds['south']
        
        while lat <= bounds['north']:
            lng = bounds['west']
            while lng <= bounds['east']:
                points.append((lat, lng))
                lng += lng_step
            lat += lat_step
        
        print(f"Generated {len(points)} grid points with {radius_km}km radius")
        return points
    
    def nearby_search(self, lat: float, lng: float, radius: int = 1500) -> List[str]:
        """
        Search for restaurants near a specific location.
        
        Args:
            lat: Latitude
            lng: Longitude
            radius: Search radius in meters (max 50000)
        
        Returns:
            List of place_ids
        """
        place_ids = []
        url = f"{self.base_url}/nearbysearch/json"
        
        params = {
            "location": f"{lat},{lng}",
            "radius": radius,
            "type": "restaurant",
            "key": self.api_key
        }
        
        while True:
            response = requests.get(url, params=params)
            data = response.json()
            
            if data.get("status") not in ["OK", "ZERO_RESULTS"]:
                print(f"Search error at ({lat}, {lng}): {data.get('status')}")
                break
            
            # Collect place_ids
            for result in data.get("results", []):
                place_ids.append(result["place_id"])
            
            # Check for next page
            next_page_token = data.get("next_page_token")
            if not next_page_token:
                break
            
            # Wait before requesting next page (required by API)
            time.sleep(2)
            params = {"pagetoken": next_page_token, "key": self.api_key}
        
        return place_ids
    
    def search_all_grid_points(self, bounds: Dict[str, float], radius_km: float = 1.5) -> List[str]:
        """
        Search all grid points and return unique place_ids.
        """
        grid_points = self.generate_grid_points(bounds, radius_km)
        all_place_ids = set()
        
        for i, (lat, lng) in enumerate(grid_points, 1):
            print(f"Searching grid point {i}/{len(grid_points)}: ({lat:.4f}, {lng:.4f})")
            
            place_ids = self.nearby_search(lat, lng, radius=int(radius_km * 1000))
            new_ids = len(place_ids) - len(all_place_ids & set(place_ids))
            all_place_ids.update(place_ids)
            
            print(f"  Found {len(place_ids)} restaurants ({new_ids} new) - Total unique: {len(all_place_ids)}")
            
            # Rate limiting between grid points
            time.sleep(0.5)
        
        print(f"\n✓ Total unique restaurants found: {len(all_place_ids)}")
        return list(all_place_ids)
    
    def get_place_details(self, place_id: str) -> Dict[str, Any]:
        """
        Get detailed information for a specific place.
        """
        url = f"{self.base_url}/details/json"
        
        fields = [
            "place_id",
            "name",
            "formatted_address",
            "address_components",
            "geometry",
            "vicinity",
            "types",
            "business_status",
            "rating",
            "user_ratings_total",
            "price_level",
            "opening_hours",
            "website",
            "url",
            "reviews",
            "photos",
            "serves_beer",
            "serves_wine",
            "dine_in",
            "takeout",
            "curbside_pickup",
            "delivery",
            "reservable",
            "editorial_summary"
        ]
        
        params = {
            "place_id": place_id,
            "fields": ",".join(fields),
            "key": self.api_key
        }
        
        response = requests.get(url, params=params)
        data = response.json()
        
        if data.get("status") == "OK":
            return data.get("result", {})
        else:
            return {}
    
    def flatten_restaurant_data(self, details: Dict[str, Any]) -> Dict[str, Any]:
        """
        Flatten the nested API response into a single-level dictionary.
        """
        flattened = {
            "place_id": details.get("place_id"),
            "name": details.get("name"),
            "formatted_address": details.get("formatted_address"),
            "address_components": json.dumps(details.get("address_components", [])),
            "lat": details.get("geometry", {}).get("location", {}).get("lat"),
            "lng": details.get("geometry", {}).get("location", {}).get("lng"),
            "vicinity": details.get("vicinity"),
            "types": json.dumps(details.get("types", [])),
            "business_status": details.get("business_status"),
            "rating": details.get("rating"),
            "user_ratings_total": details.get("user_ratings_total"),
            "price_level": details.get("price_level"),
            "opening_hours_weekday_text": json.dumps(details.get("opening_hours", {}).get("weekday_text", [])),
            "opening_hours_periods": json.dumps(details.get("opening_hours", {}).get("periods", [])),
            "website": details.get("website"),
            "url": details.get("url"),
            "serves_beer": details.get("serves_beer"),
            "serves_wine": details.get("serves_wine"),
            "dine_in": details.get("dine_in"),
            "takeout": details.get("takeout"),
            "curbside_pickup": details.get("curbside_pickup"),
            "delivery": details.get("delivery"),
            "reservable": details.get("reservable"),
            "editorial_summary": details.get("editorial_summary", {}).get("overview"),
        }
        
        # Handle reviews
        reviews = details.get("reviews", [])
        flattened["reviews"] = json.dumps([{
            "rating": r.get("rating"),
            "text": r.get("text"),
            "author_name": r.get("author_name"),
            "time": r.get("time"),
            "relative_time_description": r.get("relative_time_description")
        } for r in reviews])
        
        # Handle photos
        photos = details.get("photos", [])
        flattened["photos"] = json.dumps([{
            "photo_reference": p.get("photo_reference"),
            "height": p.get("height"),
            "width": p.get("width")
        } for p in photos])
        
        return flattened

Initialize scraper

In [42]:
scraper = AustinRestaurantScraper(API_KEY)

Define Austin bounds and search with dense grid

In [18]:
# Austin metropolitan area bounding box
austin_bounds = {
    "north": 30.5168,   # North Austin/Round Rock
    "south": 30.0986,   # South Austin/Buda
    "east": -97.5684,   # East Austin
    "west": -97.9383    # West Austin/Lake Travis
}

print("=" * 60)
print("GRID-BASED RESTAURANT SEARCH")
print("=" * 60)
print(f"Search area: {austin_bounds}")
print(f"Target: ~2000 restaurants")
print("=" * 60)

# Use 1.5km radius for dense coverage
# This will create ~40-50 grid points for comprehensive coverage
place_ids = scraper.search_all_grid_points(austin_bounds, radius_km=1.5)

print(f"\n{'='*60}")
print(f"SEARCH COMPLETE: {len(place_ids)} unique restaurants found")
print(f"{'='*60}")

GRID-BASED RESTAURANT SEARCH
Search area: {'north': 30.5168, 'south': 30.0986, 'east': -97.5684, 'west': -97.9383}
Target: ~2000 restaurants
Generated 391 grid points with 1.5km radius
Searching grid point 1/391: (30.0986, -97.9383)
  Found 0 restaurants (0 new) - Total unique: 0
Searching grid point 2/391: (30.0986, -97.9164)
  Found 0 restaurants (0 new) - Total unique: 0
Searching grid point 3/391: (30.0986, -97.8946)
  Found 0 restaurants (0 new) - Total unique: 0
Searching grid point 4/391: (30.0986, -97.8727)
  Found 10 restaurants (10 new) - Total unique: 10
Searching grid point 5/391: (30.0986, -97.8508)
  Found 0 restaurants (0 new) - Total unique: 10
Searching grid point 6/391: (30.0986, -97.8289)
  Found 22 restaurants (22 new) - Total unique: 32
Searching grid point 7/391: (30.0986, -97.8071)
  Found 9 restaurants (5 new) - Total unique: 37
Searching grid point 8/391: (30.0986, -97.7852)
  Found 0 restaurants (0 new) - Total unique: 37
Searching grid point 9/391: (30.0986, 

Fetch details for all restaurants

In [43]:
print(f"\nFetching details for {len(place_ids)} restaurants...")
print("This will take a while... grab some coffee! ☕")
print("=" * 60)

all_data = []
total = len(place_ids)
errors = 0

for i, place_id in enumerate(place_ids, 1):
    if i % 50 == 0:  # Progress update every 50 restaurants
        elapsed_pct = i / total * 100
        print(f"Progress: {i}/{total} ({elapsed_pct:.1f}%) - Errors: {errors}")
    
    details = scraper.get_place_details(place_id)
    
    if details:
        flattened = scraper.flatten_restaurant_data(details)
        all_data.append(flattened)
    else:
        errors += 1
    
    # Rate limiting: avoid hitting API limits
    time.sleep(0.1)

print(f"\n{'='*60}")
print(f"DETAILS FETCH COMPLETE")
print(f"Successfully collected: {len(all_data)} restaurants")
print(f"Errors: {errors}")
print(f"{'='*60}")


Fetching details for 4475 restaurants...
This will take a while... grab some coffee! ☕
Progress: 50/4475 (1.1%) - Errors: 0
Progress: 100/4475 (2.2%) - Errors: 0
Progress: 150/4475 (3.4%) - Errors: 0
Progress: 200/4475 (4.5%) - Errors: 0
Progress: 250/4475 (5.6%) - Errors: 0
Progress: 300/4475 (6.7%) - Errors: 0
Progress: 350/4475 (7.8%) - Errors: 0
Progress: 400/4475 (8.9%) - Errors: 0
Progress: 450/4475 (10.1%) - Errors: 0
Progress: 500/4475 (11.2%) - Errors: 0
Progress: 550/4475 (12.3%) - Errors: 0
Progress: 600/4475 (13.4%) - Errors: 0
Progress: 650/4475 (14.5%) - Errors: 0
Progress: 700/4475 (15.6%) - Errors: 0
Progress: 750/4475 (16.8%) - Errors: 0
Progress: 800/4475 (17.9%) - Errors: 0
Progress: 850/4475 (19.0%) - Errors: 0
Progress: 900/4475 (20.1%) - Errors: 0
Progress: 950/4475 (21.2%) - Errors: 0
Progress: 1000/4475 (22.3%) - Errors: 0
Progress: 1050/4475 (23.5%) - Errors: 0
Progress: 1100/4475 (24.6%) - Errors: 0
Progress: 1150/4475 (25.7%) - Errors: 0
Progress: 1200/4475 

Create DataFrame

In [44]:
df = pd.DataFrame(all_data)
print(f"\nDataFrame shape: {df.shape}")
print(f"Columns: {df.shape[1]}")
print(f"Rows: {df.shape[0]}")
df.head()


DataFrame shape: (4475, 26)
Columns: 26
Rows: 4475


,place_id,name,formatted_address,address_components,lat,lng,vicinity,types,business_status,rating,...,serves_beer,serves_wine,dine_in,takeout,curbside_pickup,delivery,reservable,editorial_summary,reviews,photos
0,ChIJBcwG1ZdKW4YR4H9Pg5TYwvU,Blenders & Bowls - Westlake,"3736 Bee Caves Rd Suite 8, West Lake Hills, TX...","[{""long_name"": ""Suite 8"", ""short_name"": ""Suite...",30.280914,-97.807242,"3736 Bee Caves Rd Suite 8, West Lake Hills","[""establishment"", ""food"", ""point_of_interest"",...",OPERATIONAL,4.2,...,False,False,True,True,None,True,False,Health-conscious cafe serving smoothies & acai...,"[{""rating"": 5, ""text"": ""My favorite place to g...","[{""photo_reference"": ""AciIO2ew_BLTCnbFrsWweBvb..."
1,ChIJWW1mSSPPRIYRriBy7jZ9QgY,Tsukimi,"12400 N Interstate Hwy 35 A-111, Austin, TX 78...","[{""long_name"": ""A-111"", ""short_name"": ""A-111"",...",30.404119,-97.676021,"12400 N Interstate Hwy 35 A-111, Austin","[""establishment"", ""food"", ""point_of_interest"",...",OPERATIONAL,4.2,...,True,True,True,True,True,True,True,"Modern, low-key sushi house plating artful ren...","[{""rating"": 4, ""text"": ""The food was surprisin...","[{""photo_reference"": ""AciIO2egB6RISDznN5oMd7x3..."
2,ChIJsx9cWey1RIYRpeZVdPR4YtA,Dai Due,"2406 Manor Rd, Austin, TX 78722, USA","[{""long_name"": ""2406"", ""short_name"": ""2406"", ""...",30.284850,-97.716756,"2406 Manor Rd, Austin","[""establishment"", ""food"", ""point_of_interest"",...",OPERATIONAL,4.7,...,True,True,True,True,None,False,True,Farm-to-table fare from locally sourced ingred...,"[{""rating"": 5, ""text"": ""Amazing food; especial...","[{""photo_reference"": ""AciIO2fzIWEDXEL7EOYUT8cD..."
3,ChIJGWZOawDPRIYRwl7b7ig83Dw,El Sazón de mi Ranchito,"100 W Howard Ln, Pflugerville, TX 78660, USA","[{""long_name"": ""100"", ""short_name"": ""100"", ""ty...",30.417858,-97.657407,"100 W Howard Ln, Pflugerville","[""establishment"", ""food"", ""health"", ""point_of_...",OPERATIONAL,NaN,...,None,None,None,None,None,None,None,None,[],"[{""photo_reference"": ""AciIO2dPyDXb_zhnZ-5-4yo7..."
4,ChIJDbkBfpLORIYRDBrzDsV7VA4,The Rolling Rooster,"13717 Mopac Service Rd, Austin, TX 78727, USA","[{""long_name"": ""13717"", ""short_name"": ""13717"",...",30.437726,-97.697842,"13717 Mopac Service Rd, Austin","[""establishment"", ""food"", ""point_of_interest"",...",OPERATIONAL,4.3,...,True,True,True,True,False,True,False,Chicken & waffles highlight the menu at this c...,"[{""rating"": 2, ""text"": ""The tenders, fries and...","[{""photo_reference"": ""AciIO2cFtzHjJLhXRXRwjHFI..."


Save to CSV

In [ ]:
output_file = "data/austin_restaurants_full.csv"
df.to_csv(output_file, index=False)
print(f"✓ Saved to {output_file}")

✓ Saved to austin_restaurants_full.csv


Calculate cost

In [47]:
# Cell 10: Calculate actual cost
details_calls = len(all_data)

# Calculate nearby searches (need to regenerate grid to count)
grid_point_count = len(scraper.generate_grid_points(austin_bounds, 1.5))
nearby_searches = grid_point_count * 3  # 3 pages per grid point average

nearby_cost = nearby_searches * 0.017
details_cost = details_calls * 0.025
total_cost = nearby_cost + details_cost

print("\n" + "=" * 60)
print("COST BREAKDOWN")
print("=" * 60)
print(f"Nearby Search calls: {nearby_searches} × $0.017 = ${nearby_cost:.2f}")
print(f"Place Details calls: {details_calls} × $0.025 = ${details_cost:.2f}")
print(f"TOTAL ESTIMATED COST: ${total_cost:.2f}")
print(f"Remaining credit: ${300 - total_cost:.2f}")
print("=" * 60)

Generated 391 grid points with 1.5km radius

COST BREAKDOWN
Nearby Search calls: 1173 × $0.017 = $19.94
Place Details calls: 4475 × $0.025 = $111.88
TOTAL ESTIMATED COST: $131.82
Remaining credit: $168.18


Data Quality Overview

In [48]:
print("\n" + "=" * 60)
print("DATASET OVERVIEW")
print("=" * 60)
print(f"Total restaurants: {len(df)}")
print(f"Average rating: {df['rating'].mean():.2f}")
print(f"Total reviews: {df['user_ratings_total'].sum():,}")
print(f"\nRestaurants with ratings: {df['rating'].notna().sum()} ({df['rating'].notna().sum()/len(df)*100:.1f}%)")
print(f"Restaurants with websites: {df['website'].notna().sum()} ({df['website'].notna().sum()/len(df)*100:.1f}%)")

print(f"\nPrice level distribution:")
print(df['price_level'].value_counts().sort_index())

print(f"\nBusiness status:")
print(df['business_status'].value_counts())


DATASET OVERVIEW
Total restaurants: 4475
Average rating: 4.22
Total reviews: 3,042,131.0

Restaurants with ratings: 4233 (94.6%)
Restaurants with websites: 3579 (80.0%)

Price level distribution:
price_level
1.0    1606
2.0    1282
3.0      59
4.0      11
Name: count, dtype: int64

Business status:
business_status
OPERATIONAL           4282
CLOSED_TEMPORARILY     193
Name: count, dtype: int64


Photo and review statistics

In [51]:
def count_items(json_str):
    if pd.isna(json_str):
        return 0
    try:
        return len(json.loads(json_str))
    except:
        return 0

df['photo_count'] = df['photos'].apply(count_items)
df['review_count'] = df['reviews'].apply(count_items)

print(f"\n" + "=" * 60)
print("PHOTOS & REVIEWS")
print("=" * 60)
print(f"Total photos available: {df['photo_count'].sum():,}")
print(f"Average photos per restaurant: {df['photo_count'].mean():.1f}")
print(f"Restaurants with photos: {(df['photo_count'] > 0).sum()} ({(df['photo_count'] > 0).sum()/len(df)*100:.1f}%)")

print(f"\nTotal reviews in dataset: {df['review_count'].sum():,}")
print(f"Average reviews per restaurant: {df['review_count'].mean():.1f}")
print(f"Restaurants with reviews: {(df['review_count'] > 0).sum()} ({(df['review_count'] > 0).sum()/len(df)*100:.1f}%)")


PHOTOS & REVIEWS
Total photos available: 39,970
Average photos per restaurant: 8.9
Restaurants with photos: 4241 (94.8%)

Total reviews in dataset: 20,500
Average reviews per restaurant: 4.6
Restaurants with reviews: 4233 (94.6%)


Geographic distribution

In [50]:
print(f"\n" + "=" * 60)
print("GEOGRAPHIC COVERAGE")
print("=" * 60)
print(f"Latitude range: {df['lat'].min():.4f} to {df['lat'].max():.4f}")
print(f"Longitude range: {df['lng'].min():.4f} to {df['lng'].max():.4f}")
print("\nTop neighborhoods by restaurant count:")
print(df['vicinity'].value_counts().head(10))


GEOGRAPHIC COVERAGE
Latitude range: 30.0863 to 30.5281
Longitude range: -97.9508 to -97.5749

Top neighborhoods by restaurant count:
vicinity
Barbara Jordan Terminal, 3600 Presidential Blvd, Austin    25
Austin                                                     22
9313 Anderson Mill Rd, Austin                              17
510 E Anderson Ln, Austin                                  16
13525 Dessau Rd, Pflugerville                              11
8624 FM812, Austin                                         11
13422 Dessau Rd, Austin                                    11
2103 W Slaughter Ln, Austin                                10
7800 S 1st St, Austin                                      10
7221 McNeil Dr, Austin                                      9
Name: count, dtype: int64


In [65]:
test_reviews = df[:1]['reviews'][0]

test_reviews

'[{"rating": 5, "text": "My favorite place to grab an acai bowl! The girls who work there are so kind. Always quick and great service.", "author_name": "michelle andrea", "time": 1750213545, "relative_time_description": "4 months ago"}, {"rating": 5, "text": "Honestly fantastic. Im surprised by the current rating tbh. I got a kiwi bowl but id just say thay judging by the quality of the bowl, their ingredients are top quality too and the atmosphere was very chill too \\ud83d\\ude0a. Ill be returning for sure.", "author_name": "Ryan Michels", "time": 1753127419, "relative_time_description": "2 months ago"}, {"rating": 1, "text": "Not accepting cash in 2025 should be criminal. Not to mention there is NO signage anywhere on the front door or near the cash register that states they don\\u2019t accept cash.", "author_name": "Lin", "time": 1745540862, "relative_time_description": "5 months ago"}, {"rating": 4, "text": "Got the golden oats seasonal special. It was super yummy, warm, not too sw

In [66]:
# Parse just the first restaurant's reviews
import json

reviews_list = json.loads(df['reviews'][0])

print(f"Number of reviews: {len(reviews_list)}")
print(f"\nAll reviews:")
for i, review in enumerate(reviews_list, 1):
    print(f"\nReview {i}:")
    print(f"  Rating: {review['rating']}")
    print(f"  Author: {review['author_name']}")
    print(f"  Text: {review['text'][:100]}...")

Number of reviews: 5

All reviews:

Review 1:
  Rating: 5
  Author: michelle andrea
  Text: My favorite place to grab an acai bowl! The girls who work there are so kind. Always quick and great...

Review 2:
  Rating: 5
  Author: Ryan Michels
  Text: Honestly fantastic. Im surprised by the current rating tbh. I got a kiwi bowl but id just say thay j...

Review 3:
  Rating: 1
  Author: Lin
  Text: Not accepting cash in 2025 should be criminal. Not to mention there is NO signage anywhere on the fr...

Review 4:
  Rating: 4
  Author: Elizabeth Moulin-Franco
  Text: Got the golden oats seasonal special. It was super yummy, warm, not too sweet. I like the concept an...

Review 5:
  Rating: 3
  Author: Steven Rosenthal
  Text: Taste pretty good but 3 stars should have been 2....took over 10 min to put this in a bowl. 3-5 ppl ...


In [ ]:
test_df = pd.read_csv("data/austin_restaurants_full.csv")

In [23]:
test_df.head()

# test_df[:1]['types'].to_string(index=False)

# print(repr(test_df['types'].iloc[2]))

#unique_vals = test_df['types'][test_df['types'].str.contains('restaurant', case=False)].unique()

#print(unique_vals)

,place_id,name,formatted_address,address_components,lat,lng,vicinity,types,business_status,rating,...,serves_beer,serves_wine,dine_in,takeout,curbside_pickup,delivery,reservable,editorial_summary,reviews,photos
0,ChIJBcwG1ZdKW4YR4H9Pg5TYwvU,Blenders & Bowls - Westlake,"3736 Bee Caves Rd Suite 8, West Lake Hills, TX...","[{""long_name"": ""Suite 8"", ""short_name"": ""Suite...",30.280914,-97.807242,"3736 Bee Caves Rd Suite 8, West Lake Hills","[""establishment"", ""food"", ""point_of_interest"",...",OPERATIONAL,4.2,...,False,False,True,True,NaN,True,False,Health-conscious cafe serving smoothies & acai...,"[{""rating"": 5, ""text"": ""My favorite place to g...","[{""photo_reference"": ""AciIO2ew_BLTCnbFrsWweBvb..."
1,ChIJWW1mSSPPRIYRriBy7jZ9QgY,Tsukimi,"12400 N Interstate Hwy 35 A-111, Austin, TX 78...","[{""long_name"": ""A-111"", ""short_name"": ""A-111"",...",30.404119,-97.676021,"12400 N Interstate Hwy 35 A-111, Austin","[""establishment"", ""food"", ""point_of_interest"",...",OPERATIONAL,4.2,...,True,True,True,True,True,True,True,"Modern, low-key sushi house plating artful ren...","[{""rating"": 4, ""text"": ""The food was surprisin...","[{""photo_reference"": ""AciIO2egB6RISDznN5oMd7x3..."
2,ChIJsx9cWey1RIYRpeZVdPR4YtA,Dai Due,"2406 Manor Rd, Austin, TX 78722, USA","[{""long_name"": ""2406"", ""short_name"": ""2406"", ""...",30.284850,-97.716756,"2406 Manor Rd, Austin","[""establishment"", ""food"", ""point_of_interest"",...",OPERATIONAL,4.7,...,True,True,True,True,NaN,False,True,Farm-to-table fare from locally sourced ingred...,"[{""rating"": 5, ""text"": ""Amazing food; especial...","[{""photo_reference"": ""AciIO2fzIWEDXEL7EOYUT8cD..."
3,ChIJGWZOawDPRIYRwl7b7ig83Dw,El Sazón de mi Ranchito,"100 W Howard Ln, Pflugerville, TX 78660, USA","[{""long_name"": ""100"", ""short_name"": ""100"", ""ty...",30.417858,-97.657407,"100 W Howard Ln, Pflugerville","[""establishment"", ""food"", ""health"", ""point_of_...",OPERATIONAL,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[],"[{""photo_reference"": ""AciIO2dPyDXb_zhnZ-5-4yo7..."
4,ChIJDbkBfpLORIYRDBrzDsV7VA4,The Rolling Rooster,"13717 Mopac Service Rd, Austin, TX 78727, USA","[{""long_name"": ""13717"", ""short_name"": ""13717"",...",30.437726,-97.697842,"13717 Mopac Service Rd, Austin","[""establishment"", ""food"", ""point_of_interest"",...",OPERATIONAL,4.3,...,True,True,True,True,False,True,False,Chicken & waffles highlight the menu at this c...,"[{""rating"": 2, ""text"": ""The tenders, fries and...","[{""photo_reference"": ""AciIO2cFtzHjJLhXRXRwjHFI..."


In [25]:
import json

for i in range(10):
    name = test_df['name'][i]
    types = json.loads(test_df['types'][i])
    print(f"{name}:")
    print(f"  {types}\n")

Blenders & Bowls - Westlake:
  ['establishment', 'food', 'point_of_interest', 'restaurant']

Tsukimi:
  ['establishment', 'food', 'point_of_interest', 'restaurant']

Dai Due:
  ['establishment', 'food', 'point_of_interest', 'restaurant']

El Sazón de mi Ranchito:
  ['establishment', 'food', 'health', 'point_of_interest', 'restaurant', 'store']

The Rolling Rooster:
  ['establishment', 'food', 'point_of_interest', 'restaurant']

KFC:
  ['establishment', 'food', 'point_of_interest', 'restaurant']

Big girl on wheels:
  ['establishment', 'food', 'point_of_interest', 'restaurant']

54th Street Restaurant & Drafthouse:
  ['bar', 'establishment', 'food', 'point_of_interest', 'restaurant']

Besos Cocina & Cantina:
  ['bar', 'establishment', 'food', 'point_of_interest', 'restaurant']

Twin Lion Chinese Restaurant:
  ['establishment', 'food', 'meal_takeaway', 'point_of_interest', 'restaurant']



In [27]:
for i in range(10):
    name = test_df['name'][i]
    summary = test_df['editorial_summary'][i]
    if pd.notna(summary):
        print(f"{name}: {summary}\n")
    else:
        print(f"{name}: No summary\n")

Blenders & Bowls - Westlake: Health-conscious cafe serving smoothies & acai bowls with toppings like fruit & granola.

Tsukimi: Modern, low-key sushi house plating artful renditions of raw seafood & other Japanese fare.

Dai Due: Farm-to-table fare from locally sourced ingredients served in a rustic space with a butcher shop.

El Sazón de mi Ranchito: No summary

The Rolling Rooster: Chicken & waffles highlight the menu at this casual eatery & bar also serving fish & Southern sides.

KFC: Restaurant chain known for its buckets of fried chicken, plus combo meals & sides.

Big girl on wheels: No summary

54th Street Restaurant & Drafthouse: Chain offering a broad American menu & drinks in a relaxed setting with kitschy, eclectic decor.

Besos Cocina & Cantina: No summary

Twin Lion Chinese Restaurant: A large menu of traditional Chinese dishes, plus some Thai curries, served in a simple dining room.

